In [17]:
import folium as fl
import pandas as pd
import geopandas as g_pd
import descartes as dc
import numpy as np
import math
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, Point
import warnings
from shapely.errors import ShapelyDeprecationWarning
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)

In [3]:
## location of all ships ##
##world = fl.Map()

df_ais = pd.read_csv('TEST_QUERY_DATA.csv') #.iloc[:100]
new_df = df_ais.drop_duplicates()

place_lat = new_df['Latitude'].astype(float).to_list()
place_lon = new_df['Longitude'].astype(float).to_list()

points=[]

for i in range(len(place_lat)):
    points.append((place_lat[i], place_lon[i], i))

#print(points)
##for index, lat in enumerate(place_lat):
    ##fl.Marker([lat, place_lon[index]], popup=('Ship \n'.format(index)),icon= fl.Icon(color='blue',icon_color='white', prefix='fa', icon='ship')).add_to(world)


In [18]:
map1 = fl.Map(location=[43, 11],
               zoom_start=5)

In [19]:
##rota de cada navio com ponto de entrada e saída com base no timestamp da mensagem.
vessel_list = new_df[['FrotaID', 'Timestamp']].dropna().FrotaID.unique()

for vessel_id in vessel_list[:500]:

        points = new_df[new_df.FrotaID == vessel_id][['Latitude','Longitude','Timestamp']]
        stamp = pd.to_datetime(points['Timestamp'], errors='coerce')
        points['Timestamp'] = stamp
        ordered = points.sort_values('Timestamp')

        cv = ordered[['Latitude','Longitude']]
      
        route_pt = [tuple(x) for x in cv.to_numpy()]
        #print(route_pt)
        fl.CircleMarker(route_pt[0], popup= vessel_id, color = 'green', radius=2.0).add_to(map1)
        if len(route_pt) > 2:
         for point in route_pt[1:-1]:
            fl.CircleMarker(point, color = 'grey', radius=2.0).add_to(map1)
            if len(route_pt) > 1:
                fl.CircleMarker(route_pt[-1], popup = vessel_id, color = 'red', radius=2.0).add_to(map1)
                fl.PolyLine(route_pt, color = 'black' ).add_to(map1)
        #break
map1 



In [20]:
# ABM 1 ENTRY OF VESSEL IN AREA OF INTEREST

#df_areas = g_pd.GeoDataFrame((pd.read_csv('DDL_CURATE_FACT_dimarea.csv')).dropna())
world = fl.Map(location=[43, 11],
               zoom_start=5)
polygons = pd.DataFrame(
  {'Areas' : ['a1','a2','a3'], 
   'Latitude' : [[39.351144,39.349207,39.347807, 39.352663], [36.121267,36.121267,35.917530,35.858109], [36.740790,36.750331,36.749987,36.746747]],
   'Longitude': [[-9.377917,-9.376440,-9.373589,-9.377506],[-6.019779,-5.081239,-5.257667,-5.849967], [-6.435869,-6.427173,-6.431504,-6.432506]]})
geo_points = g_pd.GeoDataFrame(new_df,crs = 'epsg:4326', geometry=g_pd.points_from_xy(new_df.Longitude, new_df.Latitude))

def hasCrossed(vesselID, polygons):
  route = geo_points[geo_points.FrotaID == vesselID]
  for _,r in polygons.iterrows():
      pol = Polygon(zip(r['Longitude'],r['Latitude']))
      polygon = g_pd.GeoDataFrame(index= [0], crs = 'epsg:4326', geometry=[pol])
      geo_j = polygon.to_json()
      geo_j = fl.GeoJson(geo_j, style_function=lambda x: {'fillColor': 'orange'})
      fl.Popup(r['Areas']).add_to(geo_j)
      geo_j.add_to(world)
      for point in route['geometry']:
       #print(point)
        #print(pol)
        print(pol.contains(point))
        if pol.contains(point) == True:
         fl.Marker([point.y, point.x], popup=('Ship \n ' + vessel_id +'has entered area \n' + r['Areas']),icon= fl.Icon(color='blue',icon_color='white', prefix='fa', icon='ship')).add_to(world)
        #new_geo_points = route.assign(**{key: geo_points.within(geom) for key, geom in polygons.items()})
        

hasCrossed('4db3082779d1c6e299367e0603549d2c', polygons)
world   
     

False
False
False
False
False
False
False
True
True


In [ ]:
#ABM2 VESSEL IS DRIFTING
#df_ships = pd.DataFrame(pd.read_csv('DDL_CURATE_FACT_monitais.csv').drop_duplicates())
#df_ships = df_ships[['FrotaID','Latitude','Longitude','Timestamp', 'COG', 'SOG']].dropna

new = new_df[['FrotaID', 'Timestamp', 'Latitude', 'Longitude', 'Velocidade', 'Rumo']].dropna()
vessel_list = new.FrotaID.unique()


def isdrifting(minimum_drifting_duration, SOG, minimum_leeway, COG):
    for vesselId in vessel_list[:50]:
        isDrifting = False
        points = new[new.FrotaID == vesselId]
        stamp = pd.to_datetime(points['Timestamp'], errors='coerce')
        points['Timestamp'] = stamp
        ordered = points.sort_values('Timestamp')
        
        start_time = ordered['Timestamp'].iloc[0] 
        end_time = ordered['Timestamp'].iloc[-1]
        while(start_time + pd.Timedelta(minimum_drifting_duration) <= end_time):
           # for point in ordered:
                if(ordered['Velocidade'] <= SOG) | (ordered['Rumo']+ COG <= minimum_leeway):
                    isDrifting= True
    return isDrifting

#    return 0
isdrifting('2sec', 100, 200, 320)


In [ ]:
def getshape(geotype, coordinates, nome):
    try:
        a = [elem for elem in coordinates if elem != [None,None]]
        func = eval(geotype)
        s = func(a)
        return(s.wkt)
    except:
     return(None)



In [ ]:
world = fl.Map(location=[43, 11],
               zoom_start=5)

def draw_route(vessel_id):
    vessel = new_df[new_df.FrotaID == vessel_id]
    route = vessel[['Latitude','Longitude']]
    route_pt = [tuple(x) for x in route.to_numpy()]
    fl.PolyLine(route_pt, color = 'black' ).add_to(world)
        #break
    return vessel
#draw_route('f6a6cee8c614c83ad92216a0f2023d0f')
#draw_route('dcaad5d3209bf6a715ab8ac9f37c25c6')
#world

In [ ]:
##ABM3 - Vessels coliding
pd.options.mode.chained_assignment = None
ships = new_df[['FrotaID', 'Latitude','Longitude','Velocidade','Timestamp']].dropna()
vessel_list = new_df.FrotaID.unique()

def get_distance(coord1_lat, coord1_long, coord2_lat, coord2_long):
    lng_1, lat_1, lng_2, lat_2 = map(math.radians, [coord1_long, coord1_lat, coord2_long, coord2_lat])
    d_lat = lat_2 - lat_1
    d_lng = lng_2 - lng_1

    temp = (
        math.sin(d_lat / 2)**2 
      + math.cos(lat_1)
      * math.cos(lat_2)
      * math.sin(d_lng / 2)** 2
    )
    return 6371000.0 * (2 * math.atan2(math.sqrt(temp), math.sqrt(1-temp)))
#to check is vessels are approaching one another closer than an indicated distance, with speed below defined threshold
def vessel_colision ():
    max_dist = 100
    min_speed = 200
    coliding = False
    for vesselId in vessel_list[:50]:
        points = ships[ships.FrotaID == vesselId]
        stamp = pd.to_datetime(points['Timestamp'], errors='coerce')
        points['Timestamp'] = stamp
        ordered_df = pd.DataFrame(points.sort_values('Timestamp'))
        for ind in ordered_df.index[:50]:
            for ship in ships.index[:50]:
                if (ordered_df['FrotaID'][ind] != ships['FrotaID'][ship]):
                    if (get_distance(ordered_df['Latitude'][ind], ordered_df['Longitude'][ind], ships['Latitude'][ship], ships['Longitude'][ship]) < max_dist):
                        coliding = True
                        draw_route(ordered_df['FrotaID'][ind])
                        draw_route(ships['FrotaID'][ship])
                        #print(ordered_df['FrotaID'][ind])
                        #print(ships['FrotaID'][ship])
                        #print('calma')     

vessel_colision()
world

In [ ]:
#df_ships = g_pd.read_file('DDL_CURATE_FACT_dimfr.csv')
#select = g_pd.points_from_xy(df_ships['Latitude'], df_ships['Longitude'])
#ships = g_pd.GeoDataFrame( df_ships[["FrotaID"]])
#common = ships[["FrotaID"]].merge(geoShip_df[["FrotaID"]], on=["FrotaID"])   


In [ ]:
#geo_df_list = [[point.xy[1][0], point.xy[0][1]] for point in geometry]
#for coordinates in geo_df_list:
   # world.add_child(fl.Marker(location=coordinates))
#world